# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.39.0.post1 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.39.0.post1 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0.post1 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [2]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmppv47qs1w\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpb9asllrt\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp0cib_7xt\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=3\part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp57y3_mox\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=4\part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from C:\

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
752779,2,2016-01-10 00:15:31,2016-01-10 00:29:49,1,4.05,None,None,-73.982895,40.677486,-73.951241,...,1,15.0,0.5,0.5,0.3,3.26,0.0,NaN,19.56,1.0
169568,1,2016-01-26 08:12:51,2016-01-26 08:41:37,1,3.60,None,None,-73.912804,40.714542,-73.964859,...,2,19.5,0.0,0.5,0.3,0.00,0.0,NaN,20.30,1.0
1016614,2,2016-01-21 20:26:54,2016-01-21 20:29:33,5,0.52,None,None,-73.961548,40.811234,-73.964775,...,1,4.0,0.5,0.5,0.3,1.32,0.0,NaN,6.62,1.0
807157,2,2016-01-20 05:26:21,2016-01-20 05:45:40,1,8.64,None,None,-73.830269,40.759460,-73.940331,...,1,26.0,0.5,0.5,0.3,5.46,0.0,NaN,32.76,1.0
9125,2,2016-01-01 22:54:45,2016-01-02 22:25:28,1,2.02,None,None,-73.829895,40.713707,-73.830879,...,2,9.5,0.5,0.5,0.3,0.00,0.0,NaN,10.80,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752908,2,2016-12-29 14:04:00,2016-12-29 14:14:55,1,2.00,82,95,NaN,NaN,NaN,...,1,10.0,0.0,0.5,0.3,2.00,0.0,NaN,12.80,1.0
620178,1,2016-12-18 21:25:04,2016-12-18 21:31:21,1,1.60,260,129,NaN,NaN,NaN,...,2,7.0,0.5,0.5,0.3,0.00,0.0,NaN,8.30,1.0
175424,2,2016-12-15 12:54:54,2016-12-15 12:56:12,1,0.21,75,74,NaN,NaN,NaN,...,4,-3.0,0.0,-0.5,-0.3,0.00,0.0,NaN,-3.80,1.0
936202,1,2016-12-19 23:47:19,2016-12-19 23:53:34,1,1.20,225,17,NaN,NaN,NaN,...,2,6.5,0.5,0.5,0.3,0.00,0.0,NaN,7.80,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
752779,2,2016-01-10 00:15:31,2016-01-10 00:29:49,1,4.05,None,None,-73.982895,40.677486,-73.951241,...,1.0,1,10,6,0,US,0.000000e+00,1.000000,-0.781831,0.623490
169568,1,2016-01-26 08:12:51,2016-01-26 08:41:37,1,3.60,None,None,-73.912804,40.714542,-73.964859,...,1.0,1,26,1,8,US,8.660254e-01,-0.500000,0.781831,0.623490
1016614,2,2016-01-21 20:26:54,2016-01-21 20:29:33,5,0.52,None,None,-73.961548,40.811234,-73.964775,...,1.0,1,21,3,20,US,-8.660254e-01,0.500000,0.433884,-0.900969
807157,2,2016-01-20 05:26:21,2016-01-20 05:45:40,1,8.64,None,None,-73.830269,40.759460,-73.940331,...,1.0,1,20,2,5,US,9.659258e-01,0.258819,0.974928,-0.222521
9125,2,2016-01-01 22:54:45,2016-01-02 22:25:28,1,2.02,None,None,-73.829895,40.713707,-73.830879,...,1.0,1,1,4,22,US,-5.000000e-01,0.866025,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752908,2,2016-12-29 14:04:00,2016-12-29 14:14:55,1,2.00,82,95,NaN,NaN,NaN,...,1.0,12,29,3,14,US,-5.000000e-01,-0.866025,0.433884,-0.900969
620178,1,2016-12-18 21:25:04,2016-12-18 21:31:21,1,1.60,260,129,NaN,NaN,NaN,...,1.0,12,18,6,21,US,-7.071068e-01,0.707107,-0.781831,0.623490
175424,2,2016-12-15 12:54:54,2016-12-15 12:56:12,1,0.21,75,74,NaN,NaN,NaN,...,1.0,12,15,3,12,US,1.224647e-16,-1.000000,0.433884,-0.900969
936202,1,2016-12-19 23:47:19,2016-12-19 23:53:34,1,1.20,225,17,NaN,NaN,NaN,...,1.0,12,19,0,23,US,-2.588190e-01,0.965926,0.000000,1.000000


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [4]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
752779,2,2016-01-10 00:15:31,1,4.05,-73.982895,40.677486,-73.951241,40.715469,19.56,1,10,6,0,US,0.000000,1.000000,-0.781831,0.623490,2016-01-10
169568,1,2016-01-26 08:12:51,1,3.60,-73.912804,40.714542,-73.964859,40.692135,20.30,1,26,1,8,US,0.866025,-0.500000,0.781831,0.623490,2016-01-26
1016614,2,2016-01-21 20:26:54,5,0.52,-73.961548,40.811234,-73.964775,40.805164,6.62,1,21,3,20,US,-0.866025,0.500000,0.433884,-0.900969,2016-01-21
807157,2,2016-01-20 05:26:21,1,8.64,-73.830269,40.759460,-73.940331,40.744308,32.76,1,20,2,5,US,0.965926,0.258819,0.974928,-0.222521,2016-01-20
9125,2,2016-01-01 22:54:45,1,2.02,-73.829895,40.713707,-73.830879,40.693203,10.80,1,1,4,22,US,-0.500000,0.866025,-0.433884,-0.900969,2016-01-01


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [5]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpvbo6rrb4\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [6]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
4,2,2016-01-01 22:54:45,1,2.02,-73.829895,40.713707,-73.830879,40.693203,10.80,1,...,4,22,US,-0.500000,8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
11,2,2016-01-01 01:15:55,2,1.96,-73.991859,40.701584,-73.998131,40.677856,11.16,1,...,4,1,US,0.258819,9.659258e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
17,2,2016-01-01 02:40:38,5,1.93,-73.830040,40.759438,-73.843323,40.779922,9.30,1,...,4,2,US,0.500000,8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
19,2,2016-01-01 15:01:45,1,1.45,-73.955635,40.734402,-73.957817,40.717911,8.76,1,...,4,15,US,-0.707107,-7.071068e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
57,1,2016-01-01 06:04:09,1,2.70,-73.957649,40.722141,-73.931290,40.706444,14.00,1,...,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01,New Year's Day,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23858,2,2016-12-26 21:25:44,2,3.20,NaN,NaN,NaN,NaN,15.30,12,...,0,21,US,-0.707107,7.071068e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23870,2,2016-12-25 01:16:20,1,5.42,NaN,NaN,NaN,NaN,22.38,12,...,6,1,US,0.258819,9.659258e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23903,1,2016-12-25 17:50:30,1,0.90,NaN,NaN,NaN,NaN,6.30,12,...,6,17,US,-0.965926,-2.588190e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23926,2,2016-12-25 00:07:28,1,1.82,NaN,NaN,NaN,NaN,9.30,12,...,6,0,US,0.000000,1.000000e+00,-0.781831,0.623490,2016-12-25,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [7]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpesf58r4e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpesf58r4e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpesf58r4e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpesf58r4e\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-44

In [8]:
weather_df

,longitude,temperature,datetime,precipTime,usaf,precipDepth,wban,latitude
204647,-73.867,2.8,2016-01-02 03:00:00,NaN,725030,NaN,14732,40.783
204670,-73.880,-4.4,2016-01-22 13:51:00,1.0,725030,0.0,14732,40.779
204694,-73.880,5.0,2016-01-08 02:51:00,1.0,725030,0.0,14732,40.779
204701,-73.880,-1.1,2016-01-04 15:51:00,1.0,725030,0.0,14732,40.779
204715,-73.880,4.4,2016-01-01 21:51:00,1.0,725030,0.0,14732,40.779
...,...,...,...,...,...,...,...,...
1248471,-73.967,4.4,2016-12-23 13:51:00,1.0,725053,0.0,94728,40.789
1248555,-73.967,5.0,2016-12-12 13:51:00,1.0,725053,0.0,94728,40.789
1248580,-73.967,3.9,2016-12-18 07:01:00,NaN,725053,NaN,94728,40.789
1248597,-73.967,7.8,2016-12-25 00:51:00,1.0,725053,0.0,94728,40.789


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [9]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [10]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.00000
mean,1.785958,1.348833,2.816718,-73.841258,40.692861,-73.877031,40.711408,14.632402,6.500000,15.132667,3.248417,13.608083,-0.243059,-1.974122e-02,-0.079982,-0.052788,13.374375,13.923104,1067.19000
std,0.410164,1.009963,2.891857,2.724668,1.488631,2.167100,1.177931,11.409775,3.452124,8.486852,1.960790,6.676652,0.666439,7.045789e-01,0.713987,0.693600,10.337670,9.456481,2840.82058
min,1.000000,0.000000,0.000000,-115.172668,0.000000,-115.140244,0.000000,-110.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.00000
25%,2.000000,1.000000,1.040000,-73.960226,40.695110,-73.967669,40.696896,7.880000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,6.000000,6.591071,0.00000
50%,2.000000,1.000000,1.850000,-73.945774,40.746304,-73.945503,40.746773,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.125893,10.00000
75%,2.000000,1.000000,3.500000,-73.918533,40.802170,-73.912359,40.790730,17.760000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,22.944737,127.00000
max,2.000000,7.000000,47.440000,0.000000,40.961182,0.000000,41.198071,403.900000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.00000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [11]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [12]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000,1.178900e+04,11789.000000,11789.000000,11789.000000,11789.000000,11789.000000
mean,1.787174,1.349733,2.861154,-73.936756,40.747025,-73.910464,40.731394,14.677978,3.500975,15.013233,3.234286,13.580287,-0.237568,-1.808582e-02,-0.077312,-0.049317,11.946051,10.308800,197.049368
std,0.409323,1.012186,2.908133,0.041525,0.056414,1.362629,0.752621,10.407376,1.708857,8.464260,1.966139,6.684725,0.670872,7.023143e-01,0.714338,0.693827,10.122668,8.477190,1241.571217
min,1.000000,1.000000,0.010000,-74.030739,40.572845,-74.177444,0.000000,0.070000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.060000,-73.960594,40.695072,-73.968040,40.696655,8.150000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.504580,0.000000
50%,2.000000,1.000000,1.900000,-73.946327,40.746147,-73.945763,40.746357,11.300000,4.000000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,10.168182,3.000000
75%,2.000000,1.000000,3.580000,-73.918854,40.801781,-73.912704,40.789597,17.800000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,16.966923,41.000000
max,2.000000,6.000000,47.440000,-73.747253,40.879986,0.000000,41.198071,183.340000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [43]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    #model = LinearRegression()
    model=Ridge(alpha=100)
    #model = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000])
  elif algo_name == 'random_forest':
    model = RandomForestRegressor(n_estimators=5)
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [15]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [44]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression"]:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.8905273521283937
MAPE: 0.15534990904196924
RMSE: 3.255158596459213



In [33]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
    
    "model__alpha": [100,1000,10000,100000],
}

grid_search = GridSearchCV(fitPipeline, param_grid, cv=5)
grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant'))]),
                                                                         ['passengerCount',
                                                                          'tripDistance',
                                                                          'precipTime',
                                                                          'temperature',
                                                                          'precipDepth',
                                                 

In [34]:
grid_search.fit(X_train, y_train)

print("Best params:")
print(grid_search.best_params_)

Best params:
{'model__alpha': 100}


In [35]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.863
